# Sprawdzanie dokładności modelu
Plik test_data.csv jest dostępny tylko dla jury. Zawiera dane testowe - jednakowe dla wszystkich drużyn. Jest w takim samym formacie co zbiór, który otrzymaliście wraz z zadaniem, tylko już z czystymi danymi.

In [3]:
import pandas as pd
import numpy as np

In [ ]:
test_data = pd.read_csv("test_data.csv")

### Miejsce na twój kod:

### Obrobka danych trenujacych

In [4]:
import lightgbm as lgb
import re

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
pd.set_option('display.max_columns', None)

In [34]:
df = pd.read_csv('./drive/MyDrive/final_train.csv')

In [37]:
del df['Unnamed: 0']

In [10]:
tab = []
for it in df.index:
    tab.append(df.loc[it].isna().sum().sum())

In [11]:
df['Liczba bledow']=tab
df1 = df[df['Liczba bledow']<=5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
for col in df1.columns:
    if(df1[col].dtypes != object):
        df1[col] = df1[col].fillna((df1[col].mean()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
del df1['Liczba bledow']
df1.drop_duplicates(inplace = True)
df1 = df1.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Trenowanie modelu na danych treningowych obrobionych

In [14]:
best_params =  {
    'learning_rate': 0.21846825100335193,
    'max_depth': 4,
    'n_estimators': 508,
    'reg_lambda': 0.9172053528319594,
    'subsample': 0.9063808886712854,
    'random_state': 42
}

In [15]:
very_small_feats = [
    'tGravityAccmaxY',
    'tGravityAccminX',
    'subject',
    'fBodyGyromaxIndsZ',
    'tBodyAcccorrelationXY',
    'tBodyGyromaxX',
    'fBodyGyromeanFreqX'
]

In [16]:
def train_model(model, X, y):
    model.fit(X, y)
    return model

In [17]:
def predict_model(model, X):
    y_pred = model.predict(X)
    return y_pred

In [18]:
def score_model(y_test, y_pred, scoring):
    score = scoring(y_test, y_pred, average='micro')
    return score

In [19]:
def split_data_sm(df):
    X = df[very_small_feats]
    y = df['Activity']
    return X, y

In [22]:
X, y = split_data_sm(df1)

In [23]:
model = lgb.LGBMClassifier(**best_params)

In [24]:
model = train_model(model, X, y)

### Oczyszczanie danych testowych

In [ ]:
del test_data['Unnamed: 0']

In [26]:
for col in test_data.columns:
    if(test_data[col].dtypes != object):
        test_data[col] = test_data[col].fillna((test_data[col].mean()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
test_data = test_data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [29]:
X_test, y_test = split_data_sm(test_data)

In [31]:
predicted = predict_model(model, X_test)

### Sprawdzanie dokładności:
Predicted - zmienna przechowująca przewidzianą aktywność. Poniższa linijka powinna zwrócić wartość 0-1.

In [38]:
np.mean(predicted == test_data.Activity)